# Rolling FEVD estimation
## Imports

In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import pandas as pd
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mno

import networkx as nx
import datetime as dt

from sklearn.model_selection import GridSearchCV

import sys
sys.path.append('../')

import src

sys.path.append('../../kungfu/')
import kungfu as kf

/home/rubelrennfix/.virtualenvs/euraculus/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
# RUN THIS TO CREATE FILSYSTEM STRUCTURE

# import os
# path = '../data/estimated/monthly/'
# for year in range(1994, 2020):
#     os.mkdir(path+str(year))
#     for month in range(1,13):
#         os.mkdir(path+str(year)+'/'+str(month))

## Setup

In [9]:
var_grid = {'alpha': np.geomspace(1e-5, 1e0, 11),
            'lambdau': np.geomspace(1e-2, 1e2, 13),
            #'gamma': np.geomspace(1e-2, 1e2, 15),
            }

# cov_grid = {'delta': np.geomspace(0.5, 1, 11),
#             'eta': np.linspace(0, 2, 13)}

cov_grid = {'alpha': np.geomspace(1e-2, 1e0, 25)}

horizon = 21

In [10]:
desc_all = pd.DataFrame()
network_data_all = pd.DataFrame()

#### Trial Single Estimation

In [11]:
year = 2020
month = 12

In [12]:
%%time
preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 4 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    8.1s finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   55.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.8min finished


CPU times: user 20.3 s, sys: 6.78 s, total: 27.1 s
Wall time: 4min 40s


#### Rolling Estimation

In [7]:
%%time
for year in range(1994, 2021):
    for month in [3, 6, 9, 12]: #range(12, 13):
        preprocessed_data, var_cv, var, cov_cv, cov, fevd, desc, network_data = src.estimate.run_estimation_short(year, month, var_grid, cov_grid, horizon)
    
        # append data
        desc_all = desc_all.append(desc.rename(preprocessed_data.index[-1]).to_frame().T)
        network_data.index = pd.MultiIndex.from_product([[preprocessed_data.index[-1]],network_data.index], names=['sampling_date', 'permno'])
        network_data_all = network_data_all.append(network_data)

        print('DONE WITH {}/{}\n '.format(month, year))
    
        desc_all.to_csv('../data/estimated/estimation_statistics.csv')
        network_data_all.to_csv('../data/estimated/network_data.csv')

Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  4.8min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   20.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   30.0s finished


DONE WITH 3/1994
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


...

DONE WITH 9/2019
Updating penalty_weights...
Searching suitable init_lambda hyperparameter...
Fitting 12 folds for each of 143 candidates, totalling 1716 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1716 out of 1716 | elapsed:  5.1min finished


Fitting 12 folds for each of 25 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.9min finished


DONE WITH 12/2019
CPU times: user 52min 56s, sys: 10min 49s, total: 1h 3min 45s
Wall time: 15h 6min 9s
